# Developer Gamma-gamma model PyMC implementation

**Reference**:Fader, P. S., & Hardie, B. G. (2013). The Gamma-Gamma model of monetary value. February, 2, 1-9.

http://www.brucehardie.com/notes/025/gamma_gamma.pdf

In [ ]:
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pymc as pm
import pytensor.tensor as pt
import seaborn as sns

In [ ]:
from pymc_marketing import clv

## Simulate data

In [ ]:
rng = np.random.default_rng(42)

# Hyperparameters
p_true = 6.
q_true = 4.
v_true = 15.

# Number of subjects
N = 500  
# Subject level parameters
nu_true = pm.draw(pm.Gamma.dist(q_true, v_true, size=N), random_seed=rng)

# Number of observations per subject
x = rng.poisson(lam=2, size=N) + 1  
idx = np.repeat(np.arange(0, N), x)
# Observations
z = pm.draw(pm.Gamma.dist(p_true, nu_true[idx]), random_seed=rng)

In [ ]:
print(sum(x))
assert len(nu_true[idx]) == sum(x)

In [ ]:
plt.hist(z, bins=50, ec="w")
plt.xlabel("transaction value")
plt.ylabel("counts")
plt.title("Simulated data");

In [ ]:
df = pd.DataFrame(data={"z": z, "id": idx})
z_mean = df.groupby("id").mean()["z"].values
z_mean[:10]

## PyMMMC implementation

We can use the pre-built PyMMMC implementation of the Gamma-Gamma model, which also provides nice ploting and prediction methods

### Using individual transactions 𝑧

In [ ]:
model = clv.GammaGammaModelIndividual(customer_id=idx, individual_transaction_value=z)

In [ ]:
model

In [ ]:
model.fit(random_seed=rng)

In [ ]:
az.plot_posterior(model.fit_result, var_names=["p", "q", "v"], ref_val=[p_true, q_true, v_true]);

In [ ]:
expected_spend = model.expected_customer_spend(
    customer_id=idx,
    individual_transaction_value=z,
).stack(sample=("draw", "chain"))

In [ ]:
# Choose 10 lowest, median and 10 highest spending clients
selected_idxs = np.argsort(nu_true)[::-1][[10, 250, -10]]
selected_idxs

In [ ]:
sns.kdeplot(expected_spend.sel(customer_id=selected_idxs[0]), fill=True, label="low spending client")
sns.kdeplot(expected_spend.sel(customer_id=selected_idxs[1]), fill=True, label="median spending client")
sns.kdeplot(expected_spend.sel(customer_id=selected_idxs[2]), fill=True, label="high spending client")
plt.axvline(expected_spend.mean(), color="k", ls="--", label="mean")
plt.legend();

In [ ]:
new_spend = model.expected_new_customer_spend().stack(sample=("chain", "draw"))

In [ ]:
sns.kdeplot(new_spend.isel(new_customer_id=0), fill=True, label="high spending client")
plt.axvline(new_spend.mean(), color="k", ls="--", label="mean")
plt.legend();

### Using average transactions per user $\overline{z}$

In [ ]:
model = clv.GammaGammaModel(
    customer_id=idx,
    mean_transaction_value=z_mean,
    frequency=x,
)
model

In [ ]:
model.fit(random_seed=rng)

In [ ]:
az.plot_posterior(model.fit_result, var_names=["p", "q", "v"], ref_val=[p_true, q_true, v_true]);

In [ ]:
expected_spend = model.expected_customer_spend(
    customer_id=idx,
    mean_transaction_value=z_mean,
    frequency=x,
).stack(sample=("draw", "chain"))

In [ ]:
sns.kdeplot(expected_spend.sel(customer_id=selected_idxs[0]), fill=True, label="low spending client")
sns.kdeplot(expected_spend.sel(customer_id=selected_idxs[1]), fill=True, label="median spending client")
sns.kdeplot(expected_spend.sel(customer_id=selected_idxs[2]), fill=True, label="high spending client")
plt.axvline(expected_spend.mean(), color="k", ls="--", label="mean")
plt.legend();

In [ ]:
new_spend = model.expected_new_customer_spend().stack(sample=("chain", "draw"))

In [ ]:
sns.kdeplot(new_spend.isel(new_customer_id=0), fill=True, label="high spending client")
plt.axvline(new_spend.mean(), color="k", ls="--", label="mean")
plt.legend();

## Manual PyMC implementations

We show how the Gamma-Gamma model can be implemented by hand using PyMC. This clarifies how the model can be modified or extended to include more prior information or additional structure.

### Gamma-Gamma model conditioned on individual transactions $z$

In [ ]:
with pm.Model() as m1:
    p = pm.HalfFlat("p")
    q = pm.HalfFlat("q")
    v = pm.HalfFlat("v")
    
    nu = pm.Gamma("nu", q, v, size=N)
    pm.Gamma("z", p, nu[idx], observed=z)

    pm.Deterministic("mean_spend", p / nu)
    
    trace1 = pm.sample(random_seed=rng)

In [ ]:
az.summary(trace1, var_names=["p", "q", "v"])

In [ ]:
az.plot_posterior(trace1, var_names=["p", "q", "v"], ref_val=[p_true, q_true, v_true]);

### Gamma-gamma model conditioned on average transactions per user $\overline{z}$

This fails to sample because the model contains "nearly" two independent parameters per observation. For more details check this [Discourse topic](https://discourse.pymc.io/t/gamma-model-sampling-much-worse-when-observation-summaries-are-used-instead-of-individual-observations/10444)

In [ ]:
with pm.Model() as m2:
    p = pm.HalfFlat("p")
    q = pm.HalfFlat("q")
    v = pm.HalfFlat("v")

    nu = pm.Gamma("nu", q, v, size=N)
    # We use the convolution properties of the gamma distribution to model
    # the mean of multiple transaction using the parameters of individual
    # transactions
    pm.Gamma("z_mean", p*x, nu*x, observed=z_mean)
    
    trace2 = pm.sample(random_seed=rng)

In [ ]:
az.summary(trace2, var_names=["p", "q", "v"])

In [ ]:
az.plot_posterior(trace2, var_names=["p", "q", "v"], ref_val=[p_true, q_true, v_true]);

### Gamma-Gamma model conditioned on average transaction per user with $\nu$ marginalized

In [ ]:
with pm.Model() as m3:
    p = pm.HalfFlat("p")
    q = pm.HalfFlat("q")
    v = pm.HalfFlat("v")

    # Likelihood of z_mean, marginalizing over nu
    likelihood = pm.Potential(
        "likelihood", 
        (
            pt.gammaln(p * x + q)
            - pt.gammaln(p * x)
            - pt.gammaln(q)
            + q * pt.log(v)
            + (p * x - 1) * pt.log(z_mean)
            + (p * x) * pt.log(x)
            - (p * x + q) * pt.log(x * z_mean + v)
        ),
    )

    # Closed form solution posterior individual nu
    nu = pm.Deterministic("nu", pm.Gamma.dist(p * x + q, v + x * z_mean))
    pm.Deterministic("mean_spend", p / nu)
    
    trace3 = pm.sample(random_seed=rng)

In [ ]:
az.summary(trace3, var_names=["p", "q", "v"])

In [ ]:
az.plot_posterior(trace3, var_names=["p", "q", "v"], ref_val=[p_true, q_true, v_true]);